In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#CONDA ENVIRONEMNT P312

import os
#add to system path
os.environ["OPENCV_IO_ENABLE_OPENEXR"] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from metrics_helpers import read_image, pre_hdr_p3, align_hdr_pred_to_gt, psnr, vsi, piqe, lpips, hdr_vdp3, pu, reinhard_tonemap
import os
import numpy as np


from metrics_helpers import initialize_fid, initialize_fvd
from metrics_helpers import compute_fid, fid_update, compute_fvd, fvd_update





/home/tedlasai/miniconda3/envs/p312/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:

pred_dir = "/home/tedlasai/hdrvideo/evaluations/ours_stuttgart/under/"
gt_dir =  "/home/tedlasai/hdrvideo/evaluations/stuttgart/hdr/"
# video_paths = sorted(os.listdir(pred_dir))
video_paths = ["bistro_01","bistro_02"]

reinhard_fvd_metric = initialize_fvd()
reinhard_fid_metric = initialize_fid()

pu_fvd_metric = initialize_fvd()
pu_fid_metric = initialize_fid()

psnr_scores = []
vsi_scores = []
piqe_scores = []
lpips_scores = []

for video_path in video_paths:
    pred_video_dir = os.path.join(pred_dir, video_path)
    gt_video_dir = os.path.join(gt_dir, video_path)
    im_paths = sorted(os.listdir(pred_video_dir))[:4]

    reinhard_preds = []
    reinhard_gts = []
    pu_preds = []
    pu_gts = []
    for im_path in im_paths:
        pred_im_path = os.path.join(pred_video_dir, im_path)
        gt_im_path = os.path.join(gt_video_dir, im_path)
        cv2_hdr_pred = read_image(pred_im_path)
        cv2_hdr_gt = read_image(gt_im_path)
        cv2_hdr_gt = pre_hdr_p3(cv2_hdr_gt)
        cv2_hdr_pred,cv2_hdr_gt,_ = align_hdr_pred_to_gt(cv2_hdr_pred, cv2_hdr_gt)

        # import matplotlib.pyplot as plt

        # fig, axes = plt.subplots(1, 2, figsize=(10, 4), sharey=True)

        # axes[0].hist(cv2_hdr_gt.flatten(), bins=100, range=(0, 1000))
        # axes[0].set_title("GT")

        # axes[1].hist(cv2_hdr_pred.flatten(), bins=100, range=(0, 1000))
        # axes[1].set_title("Pred")

        # plt.tight_layout()
        # plt.show()

        # fig, axes = plt.subplots(1, 2, figsize=(10, 5))

        # axes[0].imshow(cv2_hdr_gt / np.max(cv2_hdr_gt))
        # axes[0].set_title("GT")
        # axes[0].axis("off")

        # axes[1].imshow(cv2_hdr_pred / np.max(cv2_hdr_pred))
        # axes[1].set_title("Pred")
        # axes[1].axis("off")

        # plt.tight_layout()
        # plt.show()

        jod = hdr_vdp3(cv2_hdr_pred, cv2_hdr_gt)

        pu_pred, pu_gt = pu(cv2_hdr_pred), pu(cv2_hdr_gt)
        pu_preds.append(pu_pred/np.max(pu_gt))
        pu_gts.append(pu_gt/np.max(pu_gt))

        reinhard_pred, reinhard_gt = reinhard_tonemap(cv2_hdr_pred), reinhard_tonemap(cv2_hdr_gt)
        reinhard_preds.append(reinhard_pred)
        reinhard_gts.append(reinhard_gt)


        pu_psnr = psnr(pu_pred, pu_gt)
        psnr_scores.append(pu_psnr)
        print("PU-PSNR:", pu_psnr)

        pu_vsi = vsi(pu_pred, pu_gt)
        vsi_scores.append(pu_vsi)   
        print("PU-VSI:", pu_vsi)

        pu_piqe = piqe(pu_pred)
        piqe_scores.append(pu_piqe)
        print("PIQE Score:", pu_piqe)

        pu_lpips = lpips(reinhard_pred, reinhard_gt)
        lpips_scores.append(pu_lpips)
        print("LPIPS Score:", pu_lpips)


        # fid_update(reinhard_pred, reinhard_gt, reinhard_fid_metric)
        # fid_update(pu_pred, pu_gt, pu_fid_metric)
        
    fvd_update(reinhard_preds, reinhard_gts, reinhard_fvd_metric)
    fvd_update(pu_preds, pu_gts, pu_fvd_metric)

# print("R-FID Score:", compute_fid(reinhard_fid_metric))
print("R-FVD Score:", compute_fvd(reinhard_fvd_metric))
# print("PU-FID Score:", compute_fid(pu_fid_metric))
print("PU-FVD Score:", compute_fvd(pu_fvd_metric))

print("Average PU-PSNR:", np.mean(np.array(psnr_scores)))
print("Average PU-VSI:", np.mean(np.array(vsi_scores)))
print("Average PIQE:", np.mean(np.array(piqe_scores)))
print("Average LPIPS:", np.mean(np.array(lpips_scores)))

    #add fid and fvd (then loop over videos, computer, save to csv)




Loading videomae model ...
Loading videomae model ...
PU-PSNR: 23.015549
PU-VSI: 0.9831276225897186
PIQE Score: 22.94376
LPIPS Score: 0.1962708830833435
PU-PSNR: 23.053814
PU-VSI: 0.983399357102978
PIQE Score: 21.128637
LPIPS Score: 0.1977754533290863
PU-PSNR: 23.010555
PU-VSI: 0.9832050596010347
PIQE Score: 24.785967
LPIPS Score: 0.2003951370716095
PU-PSNR: 23.04801
PU-VSI: 0.983188014986581
PIQE Score: 23.179743
LPIPS Score: 0.20237594842910767
PU-PSNR: 21.899685
PU-VSI: 0.9880647102286683
PIQE Score: 45.35233
LPIPS Score: 0.09521180391311646
PU-PSNR: 21.835318
PU-VSI: 0.9878661738812461
PIQE Score: 41.74559
LPIPS Score: 0.08944909274578094
PU-PSNR: 21.783247
PU-VSI: 0.9877924892629196
PIQE Score: 46.167854
LPIPS Score: 0.08692405372858047
PU-PSNR: 21.78904
PU-VSI: 0.9878509877094722
PIQE Score: 45.83061
LPIPS Score: 0.08745228499174118


/home/tedlasai/miniconda3/envs/p312/lib/python3.12/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: The ``compute`` method of metric FrechetInceptionDistance was called before the ``update`` method which may lead to errors, as metric states have not yet been updated.
  warnings.warn(*args, **kwargs)  # noqa: B028


RuntimeError: More than one sample is required for both the real and fake distributed to compute FID